In [0]:
import numpy as np

In [0]:
def sigmoid(z, grad=False):
    if grad:
        return sigmoid(z) * (1 - sigmoid(z))
    return 1 / (1 + np.exp(-z))

def relu(z, grad=False):
    if grad:
        return np.where(z <= 0, 0, 1)
    return np.where(z <= 0, 0, z)

def tanh(z, grad=False):
    if grad:
        return 1 - (tanh(z)) ** 2
    return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))

def leaky_relu(z, slope, grad=False):
    if grad:
        return np.where(z <= 0, slope, 1)
    return np.where(z <= 0, slope*z, z)

def log_loss(a, y, grad=False):
    if grad:
        return (a - y) / (a * (1 - a))
    return np.where(y == 0, -np.log(1 - a), -np.log(a))

def init_weights(random=True):
    global x, first_hidden, second_hidden
    if random:
        W_1 = np.random.random((x_dim, first_hidden))           # первая матрица весов, соединяющая 0 и 1 слой
        b_1 = np.random.random((1, first_hidden))             # первый вектор смещений, соединяющая 0 и 1 слой
        W_2 = np.random.random((first_hidden, second_hidden))   # вторая матрица весов, соединяющая 1 и 2 слой
        b_2 = np.random.random((1, second_hidden))            # второй вектор весов, соединяющая 1 и 2 слой
        return W_1, b_1, W_2, b_2
    W_1 = np.zeros((x_dim, first_hidden))      
    b_1 = np.zeros((1, first_hidden))          
    W_2 = np.zeros((first_hidden, second_hidden)) 
    b_2 = np.zeros((1, second_hidden))            
    return W_1, b_1, W_2, b_2


In [0]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

# загрузка данных для бинарной классификации
x = data['data']
y = data['target'].reshape(-1, 1)
x = (x - np.mean(x)) / np.std(x)

first_hidden = 64     # 4 нейрона в первом скрытом
second_hidden = 1    # 1 нейрон на выходе

x_n, x_dim = x.shape

W_1, b_1, W_2, b_2 = init_weights(random=False)

In [0]:
# Юнит-тесты для проверки правильной спецификации параметров

assert (x.shape == (x_n, x_dim))
assert (y.shape == (x_n, 1))
assert (W_1.shape == (x_dim, first_hidden))
assert (b_1.shape == (1, first_hidden))
assert (W_2.shape == (first_hidden, second_hidden))
assert (b_2.shape == (1, second_hidden))

In [51]:
W_1, b_1, W_2, b_2 = init_weights(random=True) 

PRINT_EVERY = 10
EPOCHS = 100
lr = 0.1
slope = 0.05
activation = 'sigmoid'


for i in range(EPOCHS):
    # forward pass
    z_1 = x @ W_1 + b_1
    a_1 = eval(activation)(z_1)
    z_2 = a_1 @ W_2 + b_2
    a_2 = sigmoid(z_2)
    loss = log_loss(a_2, y).mean()
    if i % PRINT_EVERY == 0:
        print('Log-loss: {:.3f}, Iter: {}'.format(loss, i))

    # backward pass
    da_2 = log_loss(a_2, y, grad=True)
    dz_2 = da_2 * sigmoid(z_2, grad=True)
    dw_2 = (1 / x_n) * a_1.T @ dz_2
    db_2 = dz_2.mean(axis=0, keepdims=True)

    dw_1 = (1 / x_n) * x.T @ (W_2.T * dz_2 * eval(activation)(z_1, grad=True))
    db_1 = (W_2.T * dz_2 * eval(activation)(z_1, grad=True)).mean(axis=0, keepdims=True)
    
    W_1 -= lr * dw_1
    W_2 -= lr * dw_2
    b_1 -= lr * db_1
    b_2 -= lr * db_2



Log-loss: 9.033, Iter: 0
Log-loss: 2.629, Iter: 10
Log-loss: 0.652, Iter: 20
Log-loss: 0.522, Iter: 30
Log-loss: 0.446, Iter: 40
Log-loss: 0.399, Iter: 50
Log-loss: 0.367, Iter: 60
Log-loss: 0.343, Iter: 70
Log-loss: 0.325, Iter: 80
Log-loss: 0.311, Iter: 90
